# Per session and per user analysis

Analysis of users.

# Table of Contents

1.  [Preparation](#preparation)

2.  [Constants](#constants)

3.  [Functions](#functions)

# Preparation
<a id=preparation />

In [ ]:
%run "../Functions/1. Google form analysis.ipynb"

# Constants
<a id=constants />

In [ ]:
perSessionRelevantColumns = ['sessionId', 'serverTime', 'section']

reachEvents = df152[df152['type']=='reach'].loc[:,perSessionRelevantColumns]
deathEvents = df152[df152['type']=='death'].loc[:,perSessionRelevantColumns]

timedSectionsIndex = [
                        'tutorial1.Checkpoint00',
                        'tutorial1.Checkpoint01',
                        'tutorial1.Checkpoint02',
                        'tutorial1.Checkpoint03',
                        'tutorial1.Checkpoint04',
                        'tutorial1.Checkpoint05',
                        'tutorial1.Checkpoint06',
                        'tutorial1.Checkpoint07',
                        'tutorial1.Checkpoint08',
                        'tutorial1.Checkpoint09',
                        'tutorial1.Checkpoint10',
                        'tutorial1.Checkpoint11',
                        'tutorial1.Checkpoint12',
                        'tutorial1.Checkpoint13',
                        'tutorial1.Checkpoint14',
                     ]
timedSectionsReachedColumns = ['serverTime', 'firstReached', 'firstCompletionDuration']
timedSectionsDeathsColumns = ['deathsCount']
eventSectionsCountColumns = ['section', 'count']
eventSectionsColumns = ['count']

# Functions
<a id=functions />

In [ ]:
## Comparison between game and Google form performance

In [ ]:
# Returns a given session's checkpoints, the first server time at which they were reached, and completion time
def getCheckpointsTimes( sessionId ):
    perSession = reachEvents[reachEvents['sessionId']==sessionId]
    perSession = perSession[perSession['section'].str.startswith('tutorial', na=False)]

    timedSections = pd.DataFrame(data=0, columns=timedSectionsReachedColumns,index=timedSectionsIndex)
    timedSections["serverTime"] = perSession.groupby("section").agg({ "serverTime": np.min })
    timedSections["firstReached"] = pd.to_datetime(timedSections["serverTime"])
    timedSections["firstCompletionDuration"] = timedSections["firstReached"].diff()
    
    if(timedSections.loc["tutorial1.Checkpoint00","firstReached"] != pd.Timestamp(0)):
        timedSections.loc["tutorial1.Checkpoint00","firstCompletionDuration"] = \
        pd.Timedelta(0)
    
    timedSections["serverTime"] = timedSections["serverTime"].fillna('')
    timedSections["firstReached"] = timedSections["firstReached"].fillna(pd.Timestamp(0))
    timedSections["firstCompletionDuration"] = timedSections["firstCompletionDuration"].fillna(pd.Timedelta.max)
    
    return timedSections

In [ ]:
# Returns a given user's checkpoints, the first server time at which they were reached, and completion time
def getCheckpointsTimesUser( _userId, _sessionsList = [] ):
    # List of associated sessions
    if( len(_sessionsList) == 0):
        _sessionsList = getAllSessionsOfUser( df152, _userId, True )

    # Call getCheckpointsTimes on all sessions associated with user,
    # then merge by taking oldest checkpoint completion
    _timedSections = pd.DataFrame(data=0, columns=timedSectionsReachedColumns,index=timedSectionsIndex)
    _timedSections["serverTime"] = ''
    _timedSections["firstReached"] = pd.Timestamp(0)
    _timedSections["firstCompletionDuration"] = pd.Timedelta.max

    # merge
    # for each checkpoint reached, update if necessary
    for _sessionId in _sessionsList['sessionId']:        
        _thisSessionTimes = getCheckpointsTimes( _sessionId )

        for _checkpointName in _thisSessionTimes.index:
            if ((_thisSessionTimes.loc[_checkpointName, 'firstReached'] != pd.Timestamp(0))
                and
                ((_timedSections.loc[_checkpointName, 'firstReached'] == pd.Timestamp(0))
                or (_timedSections.loc[_checkpointName, 'firstReached'] > _thisSessionTimes.loc[_checkpointName, 'firstReached']))
               ):
                _timedSections.loc[_checkpointName, 'serverTime'] = _thisSessionTimes.loc[_checkpointName, 'serverTime']
                _timedSections.loc[_checkpointName, 'firstReached'] = _thisSessionTimes.loc[_checkpointName, 'firstReached']
                _timedSections.loc[_checkpointName, 'firstCompletionDuration'] = _thisSessionTimes.loc[_checkpointName, 'firstCompletionDuration']

    return _timedSections

In [ ]:
# Returns a given session's checkpoints, and death count
def getDeaths( sessionId ):
    perSession = deathEvents[deathEvents['sessionId']==sessionId]
    perSession = perSession[perSession['section'].str.startswith('tutorial', na=False)]
    deathsSections = perSession.groupby("section").size().reset_index(name='deathsCount')
    return deathsSections

In [ ]:
def getDeathsUser( userId ):
    #print("getDeathsUser(" + str(userId) + ")")
    
    # List of associated sessions
    sessionsList = getAllSessionsOfUser( df152, userId, True )
    #print("sessionsList=" + str(sessionsList))
    
    # Call getDeaths on all sessions associated with user,
    # then merge by adding
    deathsSections = pd.DataFrame(0, columns=timedSectionsDeathsColumns,index=timedSectionsIndex)
    
    for sessionId in sessionsList['sessionId']:        
        #print("processing user " + str(userId) + " with session " + str(sessionId))
        deaths = getDeaths( sessionId )
        
        # merge
        # for each checkpoint reached, update if necessary
        for index in deaths.index:
            #print("index=" + str(index))
            checkpointName = deaths['section'][index]
            #print("checkpointName=" + str(checkpointName))
            #print("deaths['deathsCount']["+str(index)+"]=" + str(deaths['deathsCount'][index]))
            
            deathsSections['deathsCount'][checkpointName] = deathsSections['deathsCount'][checkpointName] + deaths['deathsCount'][index]
    
    return deathsSections

## Craft events: equip, unequip, add, remove

#### event-column association

    equip device = 'add' + customData.device
    
    unequip device = 'remove' + customData.device
    
    add brick = 'add' + customData.biobrick
    
    remove brick = 'remove' + customData.biobrick

In [ ]:
# Static data
# craftEventsColumns = pd.DataFrame(
#    index=list(range(4)),
#    data={
#        'eventCode' : pd.Categorical(["equip","unequip","add","remove"]),
#        'eventType' : pd.Categorical(["add","remove","add","remove"]),
#        'column' : pd.Categorical(["customData.device","customData.device","customData.biobrick","customData.biobrick"]),
#    }
#)
#craftEventsColumns

In [ ]:
# Static data
craftEventCodes = list(["equip","unequip","add","remove"])
craftEventsColumns = pd.DataFrame(
    index=craftEventCodes,
    data={
        'eventType' : pd.Categorical(["add","remove","add","remove"]),
        'column' : pd.Categorical(["customData.device","customData.device","customData.biobrick","customData.biobrick"]),
    }
)

In [ ]:
# Returns a given session's checkpoints, and event count
# eventCode != RedMetrics' eventType
# eventCodes are craftEventsColumns' index
def getSectionsCraftEvents( eventCode, sessionId ):
    #print("getSectionsCraftEvents(" + str(eventCode) + "," + str(sessionId) + ")")
    sectionsEvents = pd.DataFrame(0, columns=eventSectionsCountColumns, index=range(0))
    if eventCode in craftEventCodes:
        eventType = craftEventsColumns['eventType'][eventCode]
        events = df152[df152['type']==eventType]
        events = events[events[craftEventsColumns['column'][eventCode]].notnull()]
        #print("getSectionsCraftEvents(" + str(eventCode) + "," + str(sessionId) + "): #events=" + str(len(events)))
        #print("events=" + str(events.head()))
        events = events.loc[:,perSessionRelevantColumns]
        perSession = events[events['sessionId']==sessionId]
        perSession = perSession[perSession['section'].str.startswith('tutorial', na=False)]
        sectionsEvents = perSession.groupby("section").size().reset_index(name='count')
    else:
        print("incorrect event code '" + eventCode + "'")
    return sectionsEvents

In [ ]:
# eventCode != RedMetrics' eventType
# eventCodes are craftEventsColumns' index
def getUserSectionsCraftEvents( eventCode, userId, sessionsList = [] ):
    #print("getUserSectionsCraftEvents(" + str(eventCode) + "," + str(userId) + ")")
    
    # Call getSectionsEvents on all sessions associated with user,
    # then merge by adding
    userSectionsEvents = pd.DataFrame(0, columns=eventSectionsColumns,index=timedSectionsIndex)
    
    if eventCode in craftEventCodes:
        # List of associated sessions
        if(len(sessionsList) == 0):
            sessionsList = getAllSessionsOfUser( df152, userId, True )
        #print("sessionsList=" + str(sessionsList))
    
        for sessionId in sessionsList['sessionId']:
            sessionSectionsEvents = getSectionsCraftEvents( eventCode, sessionId )

            # merge
            # for each checkpoint reached, update if necessary
            for index in sessionSectionsEvents.index:
                checkpointName = sessionSectionsEvents['section'][index]
                userSectionsEvents['count'][checkpointName] = userSectionsEvents['count'][checkpointName] + sessionSectionsEvents['count'][index]
    else:
        print("incorrect event code '" + eventCode + "'")
    return userSectionsEvents

In [ ]:
# eventCode != RedMetrics' eventType
# eventCodes are craftEventsColumns' index
def getUserSectionsCraftEventsTotal( eventCode, userId, sessionsList = [] ):
    #print("getUserSectionsCraftEventsTotal(" + str(eventCode) + "," + str(userId) + ")")
    events = getUserSectionsCraftEvents( eventCode, userId, sessionsList )
    return events.values.sum()

#### craft, no section

In [ ]:
# eventCode != RedMetrics' eventType
# eventCodes are craftEventsColumns' index
def getUserCraftEventsTotal( eventCode, userId, sessionsList=[] ):
    if(len(sessionsList) == 0):
        sessionsList = getAllSessionsOfUser( df152, userId, True )

    if eventCode in craftEventCodes:
        eventType = craftEventsColumns['eventType'][eventCode]
        events = df152[df152['type']==eventType]
        events = events[events[craftEventsColumns['column'][eventCode]].notnull()]
        perSession = events[events['sessionId'].isin(sessionsList['sessionId'])]
        return len(perSession)
    else:
        print("incorrect event code '" + eventCode + "'")
        return 0

## Generic functions

### Generic count

In [ ]:
# Returns a given session's checkpoints, and event count
def getSectionsEvents( eventType, sessionId ):
    events = df152[df152['type']==eventType].loc[:,perSessionRelevantColumns]
    perSession = events[events['sessionId']==sessionId]
    perSession = perSession[perSession['section'].str.startswith('tutorial', na=False)]
    sectionsEvents = perSession.groupby("section").size().reset_index(name='count')
    return sectionsEvents

In [ ]:
def getUserSectionsEvents( eventType, userId, sessionsList=[] ):
    
    # List of associated sessions
    if(len(sessionsList) == 0):
        sessionsList = getAllSessionsOfUser( df152, userId, True )
    
    # Call getSectionsEvents on all sessions associated with user,
    # then merge by adding
    userSectionsEvents = pd.DataFrame(0, columns=eventSectionsColumns,index=timedSectionsIndex)
    
    for sessionId in sessionsList['sessionId']:
        sessionSectionsEvents = getSectionsEvents( eventType, sessionId )
        
        # merge
        # for each checkpoint reached, update if necessary
        for index in sessionSectionsEvents.index:
            checkpointName = sessionSectionsEvents['section'][index]
            userSectionsEvents['count'][checkpointName] = userSectionsEvents['count'][checkpointName] + sessionSectionsEvents['count'][index]
    
    return userSectionsEvents

In [ ]:
def getUserSectionsEventsTotal( eventType, userId, sessionsList=[] ):
    events = getUserSectionsEvents( eventType, userId, sessionsList )
    return events.values.sum()

### No section events

In [ ]:
def getUserEventsTotal( eventType, userId, sessionsList=[] ):
    if(len(sessionsList) == 0):
        sessionsList = getAllSessionsOfUser( df152, userId, True )

    sessionEvents = df152[df152['type']==eventType]
    perSession = sessionEvents[sessionEvents['sessionId'].isin(sessionsList['sessionId'])]
    return len(perSession)

### Other

In [ ]:
# Returns a given user's unique reached checkpoints
def getUserCheckpoints( userId ):
    #print("getUserCheckpoints(" + str(userId) + ")")

    # List of associated sessions
    sessionsList = getAllSessionsOfUser( df152, userId, True )
    #print("sessionsList=" + str(sessionsList))
    
    # List all 'reach' events with those sessionIds.
    perUser = reachEvents[reachEvents['sessionId'].isin(sessionsList[sessionsList['userId']==userId])]
    perUser = perUser[perUser['section'].str.startswith('tutorial', na=False)]
    return pd.Series(perUser['section'].unique())


def getDiscrepancyGameGForm( userId ):
    gformNonVal = getNonValidatedCheckpoints(userId)
    gformVal = getValidatedCheckpoints(userId)
    gameVal = getUserCheckpoints(userId)
    
    #sorted, unique values in series1 that are not in series2
    #np.setdiff1d(series1.values, series2.values)
    
    #user has answered questions whose answer they haven't seen in the game
    gameNotEnough = pd.Series(np.setdiff1d(gformVal.values, gameVal.values))
    
    #user has not answered questions whose answer they have seen in the game
    gformNotEnough = []
    maxGameVal = ''
    if gameVal.values.size!=0:
        gameVal.values.max()
    for nonVal in gformNonVal.values:
        if nonVal >= maxGameVal:
            gformNotEnough.append(nonVal)    
    gformNotEnough = pd.Series(gformNotEnough)
    
    return (gameNotEnough, gformNotEnough)

In [ ]:
# Static data
noSectionEventCodes = list(['start', 'selectmenu', 'switch', 'restart',\
                            'gotourl', 'gotomooc', 'configure'])

In [ ]:
simpleEvents = [
    'complete',
    'configure',
    'craft',
    'death',
    'equip',
    'unequip',
    'add',
    'remove',
    'gotomooc',
    'gotourl',
    'pickup',
    'reach',
    'restart',
    'selectmenu',
    'start',
    'switch',
    ]

# possible events: complete	configure	craft	death	equip	gotomooc	gotourl	pickup	reach	restart	selectmenu	start	switch	unequip

userDataVectorIndex = [#game
                       'sessionsCount',
                       ]

for temporality in answerTemporalities:
    userDataVectorIndex.append(scoreLabel + temporality)

userDataVectorIndex = np.concatenate( (userDataVectorIndex,
                                     simpleEvents))

In [ ]:
#allEvents = df152['type'].unique()
#allEvents = np.concatenate( simpleEvents, allEvents ).unique()
#allUserDataVectorIndex = np.concatenate( userDataVectorIndex, allEvents ).unique()

In [ ]:
# userId is RedMetrics user id
# _source is used as correction source, if we want to include answers to these questions
def getUserDataVector( userId, isRMId = True, _source = [] ):
    
    rmUserId = userId
    if(not isRMId):
        rmUserId = unsafeGetRedMetricsGUIDFormat(userId)
    
    sessionsList = getAllSessionsOfUser( df152, rmUserId, True )
    
    columnName = str(rmUserId)
    
    data = pd.DataFrame(0, columns=[columnName],index=userDataVectorIndex)
    
    score = getScore( rmUserId )
    for _temporality in score.columns:
        _score = score.loc[scoreLabel,_temporality]
        if(len(_score)>0):
            if(_temporality == 'before'):
                _score = _score[len(_score)-1]
            else:
                _score = _score[0]
        else:
            _score = np.nan
        data.loc[scoreLabel+_temporality,columnName] = _score
    
    data.loc['sessionsCount',columnName] = countSessions( df152, rmUserId, False, sessionsList)
    
    for eventName in simpleEvents:
        if eventName in craftEventCodes:
            data.loc[eventName,columnName] = getUserCraftEventsTotal(eventName, rmUserId, sessionsList)
        else:
            data.loc[eventName,columnName] = getUserEventsTotal(eventName, rmUserId, sessionsList)
        
    # time spent on each chapter
    times = getCheckpointsTimesUser(rmUserId)
    
    for chapter in timedSectionsIndex:
        data.loc[chapter,columnName] = times.loc[chapter,"firstCompletionDuration"].total_seconds()
        
    if(len(_source) != 0):
        gfUserId = unsafeGetGFormGUIDFormat(rmUserId)
        gformLine = gform[gform[localplayerguidkey] == gfUserId]
        afters = gformLine[gformLine['Temporality'] == 'after']
        if(len(afters) > 0):
            gformLine = afters.iloc[0]
        else:
            befores = gformLine[gformLine['Temporality'] == 'before']
            if(len(befores) > 0):
                gformLine = befores.iloc[len(befores)-1]
            else:
                gformLine = gformLine.iloc[len(gformLine)-1]

        # add data from the gform: binary score on each question
        gformData = getBinarized(gformLine, _source = _source)
    
        for question in gformData.index:
            data.loc[question,columnName] = gformData.loc[question]
        
    return data